## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import citipy and api key
from citipy import citipy
from config import weather_api_key, g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lavrentiya,RU,65.5833,-171.0000,31.44,90,97,9.66,overcast clouds
1,1,Dwarka,IN,22.2394,68.9678,88.27,68,0,18.61,clear sky
2,2,Ushuaia,AR,-54.8000,-68.3000,37.06,75,0,4.61,clear sky
3,3,Vao,NC,-22.6667,167.4833,74.57,75,100,25.30,overcast clouds
4,4,Katherine,AU,-14.4667,132.2667,98.92,30,40,16.11,scattered clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                  (city_data_df["Max Temp"] >= min_temp)]

new_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Dwarka,IN,22.2394,68.9678,88.27,68,0,18.61,clear sky
3,3,Vao,NC,-22.6667,167.4833,74.57,75,100,25.30,overcast clouds
5,5,Busselton,AU,-33.6500,115.3333,67.59,52,2,5.50,clear sky
6,6,Atuona,PF,-9.8000,-139.0333,79.27,75,100,15.73,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,76.95,69,100,16.69,overcast clouds
10,10,Bure,ET,10.7000,37.0667,74.91,23,15,0.72,few clouds
14,14,Cabo Rojo,PR,18.0866,-67.1457,77.68,80,82,6.69,broken clouds
17,17,Binzhou,CN,37.3667,118.0167,65.62,15,99,10.87,overcast clouds
18,18,Butaritari,KI,3.0707,172.7902,81.30,80,37,10.13,light rain
20,20,San Patricio,US,28.0170,-97.5169,77.09,90,4,13.27,clear sky


In [10]:
# 4a. Determine if there are any empty rows.
new_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_cities_df.replace('',np.nan, inplace=True)

/Users/jakegarrison/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [12]:
new_cities_df.dropna()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Dwarka,IN,22.2394,68.9678,88.27,68,0,18.61,clear sky
3,3,Vao,NC,-22.6667,167.4833,74.57,75,100,25.30,overcast clouds
5,5,Busselton,AU,-33.6500,115.3333,67.59,52,2,5.50,clear sky
6,6,Atuona,PF,-9.8000,-139.0333,79.27,75,100,15.73,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,76.95,69,100,16.69,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
614,614,Abu Kamal,SY,34.4506,40.9171,84.07,27,30,10.04,scattered clouds
616,616,Camana,PE,-16.6228,-72.7111,65.50,87,88,7.96,overcast clouds
618,618,Beloha,MG,-25.1667,45.0500,73.36,50,0,8.48,clear sky
619,619,Codrington,AU,-38.2667,141.9667,61.23,57,98,16.17,overcast clouds


In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dwarka,IN,88.27,clear sky,22.2394,68.9678,
3,Vao,NC,74.57,overcast clouds,-22.6667,167.4833,
5,Busselton,AU,67.59,clear sky,-33.6500,115.3333,
6,Atuona,PF,79.27,overcast clouds,-9.8000,-139.0333,
8,Rikitea,PF,76.95,overcast clouds,-23.1203,-134.9692,
10,Bure,ET,74.91,few clouds,10.7000,37.0667,
14,Cabo Rojo,PR,77.68,broken clouds,18.0866,-67.1457,
17,Binzhou,CN,65.62,overcast clouds,37.3667,118.0167,
18,Butaritari,KI,81.30,light rain,3.0707,172.7902,
20,San Patricio,US,77.09,clear sky,28.0170,-97.5169,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('',np.nan, inplace=True)
hotel_df.dropna().head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Dwarka,IN,88.27,clear sky,22.2394,68.9678,Hotel Narayan Avezika Comfort
3,Vao,NC,74.57,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
5,Busselton,AU,67.59,clear sky,-33.6500,115.3333,Observatory Guest House
6,Atuona,PF,79.27,overcast clouds,-9.8000,-139.0333,Villa Enata
8,Rikitea,PF,76.95,overcast clouds,-23.1203,-134.9692,People ThankYou


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))